In [147]:
import tensorflow as tf

from tensorflow.keras.layers import Input,Embedding,Flatten, Dense,Concatenate
from tensorflow.keras.models import Model

from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt


In [148]:
data = pd.read_parquet('data_preprocessed.parquet').drop(['main_author_encoded'], axis=1) #dropping main_author_encoded as it did not lead to any improvment in performance
data = data[data.Impact != 0]
X = data.drop('Impact', axis=1)
y = data['Impact']

In [149]:
categorical_columns  = ['publisher_encoded','categories_encoded']
numerical_columns = list(set(X.columns) - set(categorical_columns))

In [150]:
scaler = StandardScaler()
X[numerical_columns] = scaler.fit_transform(X[numerical_columns])
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
X_train_cat = X_train[categorical_columns].values
X_train_num = X_train[numerical_columns].values
X_val_cat = X_val[categorical_columns].values
X_val_num = X_val[numerical_columns].values

#### Model Architecture
- using embedding layer for categorical data and flattening them
-  concatening embedding data with numerical data
- passing it to dense layer  of size 128,64,32,16
- output layer of size 1
- usnig MSE loss function




In [151]:

def create_model(cat_dims, num_dims):
    publisher_input = Input(shape=(1,), name='publisher_input')
    categories_input = Input(shape=(1,), name='categories_input')
    numeric_input = Input(shape=(num_dims,), name='numeric_input')
    
    publisher_embedding = Embedding(input_dim=cat_dims[0], output_dim=32, name='publisher_embedding')(publisher_input)
    categories_embedding = Embedding(input_dim=cat_dims[1], output_dim=32, name='categories_embedding')(categories_input)

    
    publisher_flat = Flatten()(publisher_embedding)
    categories_flat = Flatten()(categories_embedding)
    
    concatenated = Concatenate()([publisher_flat, categories_flat, numeric_input])
    x = Dense(128, activation='relu')(concatenated)
    x = Dense(64, activation='relu')(x)
    x = Dense(32, activation='relu')(x)
    x = Dense(16, activation='relu')(x)
    output = Dense(1)(x)
    
    model = Model(inputs=[publisher_input, categories_input, numeric_input], outputs=output)
    
    return model

In [152]:
cat_dims = [X['publisher_encoded'].nunique(), X['categories_encoded'].nunique()]
num_dims = len(numerical_columns)

In [153]:
model = create_model(cat_dims, num_dims)

In [154]:
model.compile(optimizer='adam', loss='mse')

In [155]:
model.summary()

Model: "model_8"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 publisher_input (InputLayer)   [(None, 1)]          0           []                               
                                                                                                  
 categories_input (InputLayer)  [(None, 1)]          0           []                               
                                                                                                  
 publisher_embedding (Embedding  (None, 1, 32)       406848      ['publisher_input[0][0]']        
 )                                                                                                
                                                                                                  
 categories_embedding (Embeddin  (None, 1, 32)       3200        ['categories_input[0][0]'] 

In [156]:
X_train_cat_list = [X_train_cat[:, i] for i in range(X_train_cat.shape[1])]
X_val_cat_list = [X_val_cat[:, i] for i in range(X_val_cat.shape[1])]

In [157]:

#using early stopping to prevent overfitting
early_stopping = EarlyStopping(
    monitor='val_loss',  
    patience=10,         
    mode='min',          
    verbose=1,        
    restore_best_weights=True  
)


In [158]:
history = model.fit(
    X_train_cat_list + [X_train_num], y_train,
    validation_data=(X_val_cat_list + [X_val_num], y_val),
    epochs=100,
    batch_size=64,
    verbose=1,
    callbacks=[early_stopping] 
)

Epoch 1/100
1735/1735 [==============================] - 7s 4ms/step - loss: 34429.4727 - val_loss: 4279.6172
Epoch 2/100
1735/1735 [==============================] - 6s 4ms/step - loss: 3832.1379 - val_loss: 3939.2104
Epoch 3/100
1735/1735 [==============================] - 6s 3ms/step - loss: 3733.1641 - val_loss: 3932.7578
Epoch 4/100
1735/1735 [==============================] - 6s 4ms/step - loss: 3675.7300 - val_loss: 4081.0537
Epoch 5/100
1735/1735 [==============================] - 6s 4ms/step - loss: 3644.7678 - val_loss: 4074.1045
Epoch 6/100
1735/1735 [==============================] - 6s 4ms/step - loss: 3602.3174 - val_loss: 3946.6421
Epoch 7/100
1735/1735 [==============================] - 6s 4ms/step - loss: 3591.1665 - val_loss: 4133.2803
Epoch 8/100
1735/1735 [==============================] - 6s 3ms/step - loss: 3578.9751 - val_loss: 3996.0586
Epoch 9/100
1735/1735 [==============================] - 6s 3ms/step - loss: 3551.9390 - val_loss: 3912.1724
Epoch 10/100
1735/

In [160]:
val_loss = model.evaluate(X_val_cat_list + [X_val_num], y_val)
print(f'Validation Loss (MSE): {val_loss:.4f}')

868/868 [==============================] - 1s 824us/step - loss: 3912.1702
Validation Loss (MSE): 3912.1702


In [161]:
predictions = model.predict(X_val_cat_list + [X_val_num])

868/868 [==============================] - 1s 647us/step


In [162]:
mape = np.mean(np.abs((y_val - predictions.flatten()) / y_val)) * 100
print(f'Mean Absolute Percentage Error (MAPE): {mape:.2f}%')

Mean Absolute Percentage Error (MAPE): 5.87%
